In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import random
import os
import torchvision.models as models

In [7]:
#定义常量和超常数
BATCH_SIZE = 64
LEARNING_RATE = 0.001
EPOCHS = 100
NOISE_RATIO = 0.4

In [8]:
# 定义一个函数来加载数据集和添加噪声
def load_dataset(noise_ratio):
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

    # 添加噪声
    num_classes = len(train_dataset.classes)
    noise_labels = np.random.choice(num_classes, int(len(train_dataset.targets) * noise_ratio), replace=True)
    for idx, label in zip(noise_labels, train_dataset.targets):
        if random.random() < (0.2 / (num_classes - 1)):
            new_label = random.randint(0, num_classes - 1)
            while new_label == label:
                new_label = random.randint(0, num_classes - 1)
            train_dataset.targets[idx] = new_label

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    return train_loader, test_loader

In [9]:
#定义噪声过滤网络的结构
class NoiseFilterNet(nn.Module):
    def __init__(self):
        super(NoiseFilterNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.bn4 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.bn4(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x
    
    #定义训练函数
    def train_model(self, train_loader, optimizer, criterion):
        self.train()
        train_loss = 0.0
        train_acc = 0.0

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.cuda(), target.cuda()

            optimizer.zero_grad()
            output = self(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            train_acc += (predicted == target).sum().item()

        train_loss /= len(train_loader.dataset)
        train_acc /= len(train_loader.dataset)

        return train_loss, train_acc

    def test_model(self, test_loader, criterion):
        self.eval()
        test_loss = 0.0
        test_acc = 0.0

        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.cuda(), target.cuda()

                output = self(data)
                loss = criterion(output, target)

                test_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                test_acc += (predicted == target).sum().item()

        test_loss /= len(test_loader.dataset)
        test_acc /= len(test_loader.dataset)

        return test_loss, test_acc

In [10]:
def main():
    # 加载数据集并添加噪声
    train_loader, test_loader = load_dataset(NOISE_RATIO)
    
    # 创建ResNet-18作为backbone
    backbone = models.resnet18(pretrained=True)
    backbone.fc = nn.Identity()  # Remove the final fully connected layer
    
    # 创建模型并将其移动到GPU上
    model = NoiseFilterNet().cuda()

    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # 训练和测试模型
    for epoch in range(EPOCHS):
        train_loss, train_acc = model.train_model(train_loader, optimizer, criterion)
        test_loss, test_acc = model.test_model(test_loader, criterion)

        print('Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.4f}, Test Loss: {:.4f}, Test Acc: {:.4f}'
              .format(epoch + 1, EPOCHS, train_loss, train_acc, test_loss, test_acc))
    
    # 保存模型
    torch.save(model.state_dict(), 'noise_filter_net.pth')

if __name__ == '__main__':
    main()


def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()

            output = model(data)
            _, predicted = torch.max(output.data, 1)

            total += target.size(0)
            correct += (predicted == target).sum().item()

    acc = 100 * correct / total
    
    return acc
if os.path.exists('noise_filter_net.pth'):
    print("File saved successfully.")
else:
    print("Failed to save the file.")

Files already downloaded and verified
Files already downloaded and verified


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/ma-user/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Epoch [1/100], Train Loss: 0.0196, Train Acc: 0.5508, Test Loss: 0.0165, Test Acc: 0.6273
Epoch [2/100], Train Loss: 0.0143, Train Acc: 0.6761, Test Loss: 0.0137, Test Acc: 0.6950
Epoch [3/100], Train Loss: 0.0125, Train Acc: 0.7182, Test Loss: 0.0134, Test Acc: 0.6943
Epoch [4/100], Train Loss: 0.0113, Train Acc: 0.7473, Test Loss: 0.0116, Test Acc: 0.7429
Epoch [5/100], Train Loss: 0.0106, Train Acc: 0.7620, Test Loss: 0.0118, Test Acc: 0.7458
Epoch [6/100], Train Loss: 0.0098, Train Acc: 0.7806, Test Loss: 0.0105, Test Acc: 0.7709
Epoch [7/100], Train Loss: 0.0092, Train Acc: 0.7961, Test Loss: 0.0100, Test Acc: 0.7780
Epoch [8/100], Train Loss: 0.0087, Train Acc: 0.8053, Test Loss: 0.0095, Test Acc: 0.7901
Epoch [9/100], Train Loss: 0.0083, Train Acc: 0.8164, Test Loss: 0.0093, Test Acc: 0.7927
Epoch [10/100], Train Loss: 0.0080, Train Acc: 0.8213, Test Loss: 0.0090, Test Acc: 0.8011
Epoch [11/100], Train Loss: 0.0075, Train Acc: 0.8313, Test Loss: 0.0095, Test Acc: 0.7900
Epoch [1

In [11]:

if __name__ == '__main__':
    # 加载测试集
    _, test_loader = load_dataset(NOISE_RATIO)

    # 加载模型并进行测试
    model = NoiseFilterNet().cuda()
    model.load_state_dict(torch.load('noise_filter_net.pth'))
    test_acc = evaluate(model, test_loader)

    print('Test Accuracy: {:.2f}%'.format(test_acc))

Files already downloaded and verified
Files already downloaded and verified
Test Accuracy: 85.59%
